En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [2]:
file_path = "farmers-protest-tweets-2021-2-4.json"

In [1]:
import os

['.ipynb_checkpoints',
 'challenge.ipynb',
 'farmers-protest-tweets-2021-2-4.json',
 'q1_memory.py',
 'q1_time.py',
 'q2_memory.py',
 'q2_time.py',
 'q3_memory.py',
 'q3_time.py']